<a href="https://colab.research.google.com/github/amotu/COVIDNet/blob/main/CovidNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install surfboard
import surfboard
from surfboard.sound import Waveform
from surfboard.feature_extraction import extract_features
import pandas
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import drive
import librosa
import librosa.display
import IPython.display as ipd
import tensorflow as tf
from tqdm import tqdm
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
%tensorflow_version 2.x

In [2]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [47]:
# Set file/audio locations
os.chdir('/content/gdrive/MyDrive/DSCI400')
new_audio_path = '/content/gdrive/MyDrive/DSCI400/Regroup Audio New/coughvid'
coughnet_audio_path = '/content/gdrive/MyDrive/DSCI400/Coughnet Audio'
coswara_audio_path = '/content/gdrive/MyDrive/DSCI400/Regroup Audio New/cough-coswara'
audio_path = new_audio_path

In [13]:
# Import functions used 
from utils.Commented.DataGenerator import DataGenerator, Data_Viz, Get_matrix_inputs
from utils.Commented.SurfboardFeatures import GenerateVectors, Reduce_Dim, Get_feature_vectors
from utils.Commented.Modeling import build_model2, build_model, build_cnn_train, build_train
from utils.Commented.Plotter import loss_plot, accuracy_plot, plot_cm
from utils.Commented.Metrics import round_using_t, get_best_threshold, get_predict, get_sen_spec
from utils.Commented.Transfer_Learning import expand_dim, expand_dim_all, make_transfer_model
from utils.Commented import Parameters

# Quick and Easy

In [12]:
from utils.Commented.DenseNet_Model import Transfer_Model
from utils.Commented.CNN_Model import CNN_Model
from utils.Commented.CNN_MLP_Model import CNN_MLP_Model

In [ ]:
# Get history of trained model
Transfer_Model_History = Transfer_Model(train = True, load = True, report = True, show_plots = True) # Train the model with loaded variables and report performance aswell as show model summary

In [ ]:
# Plot loss plot
loss_plot(Transfer_Model_History, ' Transfer ', True)

In [ ]:
# Plot accuracy plot
accuracy_plot(Transfer_Model_History, ' Transfer ', True)

In [55]:
# CNN + MLP Model
new_cnn_mlp_model = CNN_MLP_Model(train=False, load = False, report = False, show_plots = False) # Or obtain the model itself

In [61]:
# Load weights
weight_name = 'CNN_MLP_weights.h5' # Note Transfer Learning weights were not saved
weights = os.path.join(Parameters.checkpoint_loc, weight_name) # Load corresponding weights
new_cnn_mlp_model.load_weights(weights)

In [ ]:
# Load Inputs
x_train, y_train, x_test, y_test, x_val, y_val = Get_matrix_inputs(new_audio_path, coswara_audio_path, coswara_audio_path, Parameters.saved_load_loc, load=True) # If load = False, DataGenerator will run for approximately 15 min on coswara dataset and 2.25 hrs on coughvid dataset
test_data_vec, train_data_vec, val_data_vec = Get_feature_vectors(new_audio_path, coswara_audio_path, coswara_audio_path, Parameters.saved_load_loc, load=True) # If load = False, GenerateVectors will run for approximately 1 hr on coswara dataset and too long on coughvid dataset 
x_test2, x_train2, x_val2 = Reduce_Dim(Parameters.PCA_COMPONENTS, train_data_vec, test_data_vec, val_data_vec) # Reduce dimensions

In [ ]:
# Begin process of evaluation
predict, threshold, score = get_best_threshold(new_cnn_mlp_model, [x_test, x_test2], y_test, metric[1], True)
print('Threshold:', threshold, 'Score:', score)

In [42]:
# CNN Model
new_cnn_model = CNN_Model(False, False, False, False) # Third model

In [50]:
# Load weights
weight_name = 'CNN_weights.h5'
weights = os.path.join(Parameters.checkpoint_loc, weight_name) # Load corresponding weights
new_cnn_model.load_weights(weights)

In [ ]:
# Note available weights
# CNN Model: 'CNN_weights.h5'
# CNN+MLP Model: 'CNN_MLP_weights.h5'
# Transfer Model: 'Transfer_Weights_2', 'Transfer_Weights'
# Some weights were overriden and may not correspond to the weights that were used to evaluate their best performance in the final report

# Do it youself

# Prep for Modeling


## Get inputs: DataGenerator, Generate Vectors

In [ ]:
# Produces spectrogram images as well as labels for testing, training, and validation sets
x_train, y_train, x_test, y_test, x_val, y_val = Get_matrix_inputs(new_audio_path, coswara_audio_path, coswara_audio_path, Parameters.saved_load_loc, load=True)

In [ ]:
# Generates test, train, and validation feature vectors for the model
test_data_vec, train_data_vec, val_data_vec = Get_feature_vectors(new_audio_path, coswara_audio_path, coswara_audio_path, Parameters.saved_load_loc, load=True)

## DataGenerator Viz

In [ ]:
# Produces image representations of a select number of spectrograms
Data_Viz(5, 5, x_train, y_train)

## Reduce Dim of Vectors

In [ ]:
# Performs dimensionality reduction of the feature vectors
x_test2, x_train2, x_val2 = Reduce_Dim(Parameters.PCA_COMPONENTS, train_data_vec, test_data_vec, val_data_vec)

## Get Input Shapes

In [ ]:
# Get input shape 1
input_shape_1 = x_train.shape[1:]
input_shape_1

In [ ]:
# Get input shape 2
input_shape_2 = x_train2.shape[1:]
input_shape_2

In [ ]:
# Specify which metrics can be optimized
metric = {1:'sensitivity_specitivity', 2:'precision_recall', 3:'f_score'}

# CNN + MLP


In [ ]:
# Builds a model combining of a Convolutional Neural Network and a Multi-Layer Perceptron to use MFCC data as inputs to differentiate between COVID-19 positive and COVID-19 negative audio clips
new_model = None
new_model = build_model2(input_shape1=input_shape_1, input_shape2=input_shape_2, num_classes=2)

In [ ]:
# Trains CNN + Multi-Layer Perceptron model using input data from MFCCs
model_hist = build_train(new_model, [x_train, x_train2], y_train, Parameters.BATCH_SIZE, Parameters.EPOCHS, Parameters.LEARNING_RATE, check_loc=Parameters.checkpoint_loc, name='best_CNN_MLP_weights.h5', val_data = ([x_val, x_val2], y_val), show=False)

In [ ]:
# Optimize the threshold
predict, threshold, score = get_best_threshold(new_model, [x_test, x_test2], y_test, metric[1], True)
print('Threshold:', threshold, 'Score:', score)

In [ ]:
# Get prediction with threshold
y_hat = get_predict(predict, threshold)

In [ ]:
# Evaluate performance
tn, fp, fn, tp = plot_cm(y_test.reshape(-1,), y_hat, threshold)
print('true_neg:', tn)
print('false_pos:', fp)
print('false_neg:', fn)
print('true_pos:', tp)

In [ ]:
# Report Sensitivity and Specificity
Sensitivity, Specificity = get_sen_spec(y_test, y_hat)
print('Sensitivity : ', Sensitivity)
print('Specificity : ', Specificity)

In [ ]:
# Report Classification report
report = classification_report(y_test.reshape(-1,), y_hat, target_names=['0', '1'])
print(report)

In [ ]:
# Report loss plot
loss_plot(model_hist, ' CNN+SLP ', True)

In [ ]:
# Report accuracy plot
accuracy_plot(model_hist, ' CNN+SLP ', True)

# CNN (API)

In [ ]:
# Builds a Convolutional Neural Network (CNN) model 
a = None
a = build_model(input_shape_1, 2)

In [ ]:
# Trains the CNN model on MFCCs
cnn_hist = build_cnn_train(a, x_train, y_train, Parameters.BATCH_SIZE, Parameters.EPOCHS, Parameters.LEARNING_RATE, Parameters.checkpoint_loc, 'best_CNN_weights.h5', False, x_val, y_val)

In [ ]:
# Optimize threshold
predict, threshold, score = get_best_threshold(a, x_test, y_test, metric[1], True)
print('Threshold:', threshold, 'Score:', score)

In [ ]:
# Get prediction with threshold
y_hat = get_predict(predict, threshold)

In [ ]:
# Evaluate Performance
tn, fp, fn, tp = plot_cm(y_test.reshape(-1,), y_hat, threshold)
print('true_neg:', tn)
print('false_pos:', fp)
print('false_neg:', fn)
print('true_pos:', tp)

In [ ]:
# Report Classification report
report = classification_report(y_test.reshape(-1,), y_hat, target_names=['0', '1'], )
print(report)

In [ ]:
# Report Sensitivity and Specificity
Sensitivity, Specificity = get_sen_spec(y_test, y_hat)
print('Sensitivity : ', Sensitivity)
print('Specificity : ', Specificity)

In [ ]:
# Report loss plot
loss_plot(cnn_hist, ' CNN ', True)

In [ ]:
# Report accuracy plot
accuracy_plot(cnn_hist, ' CNN ', True)

# Transfer Learning API

In [ ]:
# Make 3-channel inputs
x_train_3d, x_test_3d, x_val_3d = expand_dim_all(x_train, x_test, x_val)

In [ ]:
# specify models
models = ['resnet', 'denseNet', 'vgg']

In [ ]:
# Make model on selected transfer learning model
TransferModel = make_transfer_model((512, 215, 3), models[1])

In [ ]:
# Train the model
transfer_history = build_cnn_train(TransferModel, x_train_3d, y_train, Parameters.BATCH_SIZE, Parameters.EPOCHS, Parameters.LEARNING_RATE, Parameters.checkpoint_loc, 'best_Transfer_weights.h5', False, x_val_3d, y_val)

In [ ]:
# Optimizing threshold
predict, threshold, score = get_best_threshold(TransferModel, x_test_3d, y_test, metric[1], True)
print('Threshold:', threshold, 'Score:', score)

In [ ]:
# Get prediction with threshold
y_hat = get_predict(predict, threshold)

In [ ]:
# Evaluate Performance
tn, fp, fn, tp = plot_cm(y_test.reshape(-1,), y_hat, threshold)
print('true_neg:', tn)
print('false_pos:', fp)
print('false_neg:', fn)
print('true_pos:', tp)

In [ ]:
# Report classification report
report = classification_report(y_test.reshape(-1,), y_hat, target_names=['0', '1'])
print(report)

In [ ]:
# Report Sensitivity and Specificity
Sensitivity, Specificity = get_sen_spec(y_test, y_hat)
print('Sensitivity : ', Sensitivity)
print('Specificity : ', Specificity)

In [ ]:
# Plot loss plot
loss_plot(transfer_history, ' DenseNet ', True)

In [ ]:
# Plot accuracy plot
accuracy_plot(transfer_history, ' DenseNet ', True)

# Check if best model can Generalize

In [ ]:
# Load inputs of Coughvid dataset
x_train_c, y_train_c, x_test_c, y_test_c, x_val_c, y_val_c = Get_matrix_inputs(new_audio_path, coswara_audio_path, new_audio_path, Parameters.saved_load_loc, load=True)
test_data_vec_c, train_data_vec_c, val_data_vec_c = Get_feature_vectors(new_audio_path, coswara_audio_path, new_audio_path, Parameters.saved_load_loc, load=True)
x_test2_c, x_train2_c, x_val2_c = Reduce_Dim(Parameters.PCA_COMPONENTS, train_data_vec, test_data_vec_c, val_data_vec_c)
x_train_3d_c, x_test_3d_c, x_val_3d_c = expand_dim_all(x_train_c, x_test_c, x_val_c)

In [ ]:
# Optimize threshold
predict, threshold, score = get_best_threshold(TransferModel, x_test_3d_c, y_test_c, metric[1], True)
print('Threshold:', threshold, 'Score:', score)

In [ ]:
# Get prediction with threshold
y_hat = get_predict(predict, threshold)

In [ ]:
# Evaluate Performance
tn, fp, fn, tp = plot_cm(y_test_c.reshape(-1,), y_hat, threshold)
print('true_neg:', tn)
print('false_pos:', fp)
print('false_neg:', fn)
print('true_pos:', tp)

In [ ]:
# Report Classification report
report = classification_report(y_test_c.reshape(-1,), y_hat, target_names=['0', '1'])
print(report)

In [ ]:
# Report Sensitivity and Specificity
Sensitivity, Specificity = get_sen_spec(y_test_c, y_hat)
print('Sensitivity : ', Sensitivity)
print('Specificity : ', Specificity)